In [1]:
import numpy as np
import os, sys
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import re
from datetime import time
from datetime import datetime

from scipy.io import readsav
from scipy import signal

from astropy.time import Time, TimeDelta

from sunpy import timeseries as ts
from sunpy.net import attrs as a
from sunpy.net import hek

import pandas as pd

import goesxrs_temp as gtem #from Ian's functions
import Onsets_temp as onsets #from my fucntions

# import warnings #Need to remove for future
import warnings
warnings.filterwarnings("ignore", module="sunpy.timeseries.timeseriesbase")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Just setup plot fonts
plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
                         'font.sans-serif':"Arial",'mathtext.default':"regular", 'axes.linewidth' :2})



In [2]:
data_dir = r"C:\Users\William\Documents\University\MastersProj\Data\Paper_Flares"

## Looping through all flares in directory to fetch onset info

In [3]:
#Looping through all goes-15 flares in directory
flares_df = pd.read_csv('Background Timings.csv')
# flare_data = pd.DataFrame(columns = [])

onset_length = 60 #Time of fixed onset in seconds

for index, row in flares_df.iterrows():
     try:
          #Locate Flare from downloaded files
          fl_hek_start = row['Start Time']
          fl_hek_end = row['End Time']
          fl_class = "N/A"
          flare_time = a.Time(Time(fl_hek_start, scale='utc').iso, Time(fl_hek_end,scale='utc').iso)
          day_string = flare_time.start.datetime.date().strftime("%Y%m%d")
          fl_path = os.path.join(data_dir,f'*g15*{day_string}*.nc')

          if Time(fl_hek_start).datetime.time() <= time(hour=1, minute=0, second=0):
               day_str_prev = (Time(fl_hek_start) - TimeDelta(1, format = 'jd')).datetime.date().strftime("%Y%m%d")
               fl_path_prev = os.path.join(data_dir,f'*g15*{day_str_prev}*.nc')
               g15 = ts.TimeSeries([fl_path, fl_path_prev],concatenate=True)
          elif Time(fl_hek_start).datetime.time() >= time(hour=23, minute=0, second=0):
               day_str_post = (Time(fl_hek_start)+ TimeDelta(1, format = 'jd')).datetime.date().strftime("%Y%m%d")
               fl_path_post = os.path.join(data_dir,f'*g15*{day_str_post}*.nc')
               g15 = ts.TimeSeries([fl_path, fl_path_post],concatenate=True)
          else:  
               g15 = ts.TimeSeries(fl_path, concatenate=True)

     # Load flare into time series
          g_tims = g15.index
          g_short = g15.quantity("xrsa").value 
          g_long = g15.quantity("xrsb").value
          g_short_flag = g15.quantity("xrsa_quality").value 
          g_long_flag = g15.quantity("xrsb_quality").value 

     # Create a Boolean mask where onset flag is equal to 1
          mask_long = g_long_flag == 1
          mask_short = g_short_flag == 1

          # Set the corresponding values in g_short and g_long to NaN
          g_short[mask_short] = np.nan
          g_long[mask_long] = np.nan
          df_long = pd.Series(g_long, index = pd.DatetimeIndex(g15.index))
          df_short = pd.Series(g_short, index = pd.DatetimeIndex(g15.index))

     # Printing for terminal
          print("------------------------------")
          print(f"{fl_class} Flare With Start Time: {fl_hek_start}")

     #Calculating background of non-backsubbed peak
          bcktrunc_18=df_long.truncate(flare_time.start.iso,flare_time.end.iso)
          old_peak = bcktrunc_18.idxmax()

     # Background Time Calculation Function
          #bckt, bcktrunc_054, bcktrunc_18, bckdata = background_2(df_short, df_long, start_time = Time(row['Start Time']), peak_time = Time(row['Peak Time']))
          bck_startt, bck_endt, smooth_bck, bck_tr_ts, bck_tr_vars, bck_flag = onsets.background(df_short, df_long, start_time = Time(fl_hek_start), peak_time = old_peak)
          srch_time=a.Time(Time(bck_startt, scale='utc').iso, Time(bck_endt,scale='utc').iso)
          trunc_054 = df_short.truncate(srch_time.start.iso,srch_time.end.iso)
          trunc_18 = df_long.truncate(srch_time.start.iso,srch_time.end.iso)

          print(f"Background Start: {bck_startt}, Background End: {bck_endt}")

          bck_t = trunc_054.index
          bck_short = np.mean(trunc_054)
          bck_short_std = np.std(trunc_054)
          bck_long = np.mean(trunc_18)
          bck_long_std = np.std(trunc_054)

          # print(f"long channel backg err: {bck_long_std}")
          # print(f"short channel backg err: {bck_short_std}")

     # Background Subtracting the Data
          short_backsub = df_short - bck_short
          long_backsub = df_long - bck_long

     # Calculating true peak time (using backsubbed long channel flux)
          flare_long = long_backsub.truncate(flare_time.start.iso,flare_time.end.iso)
          long_peakt = flare_long.idxmax()
          flare_short = short_backsub.truncate(flare_time.start.iso,flare_time.end.iso)
          short_peakt = flare_short.idxmax()

          long_peakfl_18 = flare_long[long_peakt]
          long_peakfl_054 = flare_short[long_peakt]
          short_peakfl_18 = flare_long[short_peakt]
          short_peakfl_054 = flare_short[short_peakt]

          true_peak = long_peakt #Varaible change for old code

     # Calculating the first acceptable point of onset using backsubbed long channel
          srch_start = Time(bck_endt) - TimeDelta(60, format = 'sec')
          trange_search = a.Time(srch_start, true_peak)
          g_long_srch = long_backsub.truncate(trange_search.start.iso,trange_search.end.iso)

          
          indexes = g_long_srch[g_long_srch >= 1e-8].index
          if len(indexes) == 0:
               raise ValueError
          else:
               first_index = indexes[0]
               
     # Onset and TEM Calculations of different onset definitions 
     # Lots of inefficient calculating going on here, but it makes it clearer to track the variables

          #onset_start = (Time(bck_endt) + (Time(true_peak)-Time(bck_endt))/8).datetime # Calculating onset start to be 1/8th into the Impulsive phase
          if first_index > bck_endt:
               onset_start = first_index
          else:
               onset_start = bck_endt

          # Calcualting the time length of the each fractional onset interval, as well as the end time of that interval
          endt_eighth = onsets.scaled_onset(bck_endt, true_peak, 8)
          timedelta_eighth = pd.to_timedelta(endt_eighth - onset_start).total_seconds()
          endt_sixth = onsets.scaled_onset(bck_endt, true_peak, 6)
          timedelta_sixth = pd.to_timedelta(endt_sixth - onset_start).total_seconds()
          endt_quarter = onsets.scaled_onset(bck_endt, true_peak, 4)
          timedelta_quarter = pd.to_timedelta(endt_quarter - onset_start).total_seconds()
          endt_third = onsets.scaled_onset(bck_endt, true_peak, 3)
          timedelta_third = pd.to_timedelta(endt_third - onset_start).total_seconds()
          endt_half = onsets.scaled_onset(bck_endt, true_peak, 2)
          timedelta_half = pd.to_timedelta(endt_half - onset_start).total_seconds()
          endt_twothirds = onsets.scaled_onset(bck_endt, true_peak, 3/2)
          timedelta_twothirds = pd.to_timedelta(endt_twothirds - onset_start).total_seconds()
          endt_threequart = onsets.scaled_onset(bck_endt, true_peak, 4/3)
          timedelta_threequart= pd.to_timedelta(endt_threequart - onset_start).total_seconds() 

          print(f"Onset start: {onset_start}, 1/4 Onset End: {endt_quarter}")

     # # Calculating errors [IMPLEMENTED TO FUNCTION NOW]
     #      #First calculate the poisson error (sqrt of flux for each raw flux bin in the range)
     #      quarter_trunc_054 = df_short.truncate(onset_start, endt_quarter)
     #      quarter_trunc_18 = df_long.truncate(onset_start, endt_quarter)
     #      quarter_trunc_054_bck = short_backsub.truncate(onset_start, endt_quarter)
     #      quarter_trunc_18_bck = long_backsub.truncate(onset_start, endt_quarter)

     #      counts_unc_054 = np.sqrt(quarter_trunc_054.values) #Uncertainty on the raw unsubbed flux measurement
     #      counts_unc_18 = np.sqrt(quarter_trunc_18.values)
     #      flux_unc_054 = np.sqrt(counts_unc_054**2 + bck_short_std) #Uncertainty on the backsubbed flux
     #      flux_unc_18 = np.sqrt(counts_unc_18**2 + back_long_std)

     #      weights_054 = 1/(flux_unc_054)**2
     #      weights_18 = 1/(flux_unc_18)**2
     #      weighted_mean_054 = np.sum(weights_054 * quarter_trunc_054_bck) / np.sum(weights_054)
     #      weighted_mean_18 = np.sum(weights_18 * quarter_trunc_18_bck) / np.sum(weights_18)
     #      err_054 = np.sqrt(np.sum(weights_054 * (quarter_trunc_054_bck - weighted_mean_054)**2) / (np.sum(weights_054) * (len(quarter_trunc_054_bck)) - 1))
     #      err_18 = np.sqrt(np.sum(weights_18 * (quarter_trunc_18_bck - weighted_mean_18)**2) / (np.sum(weights_18) * (len(quarter_trunc_18_bck)) - 1))
          
     #      print(f"Error on short channel weighted avg flux is: {err_054}; {(err_054/weighted_mean_054)*100}")
     #      print(f"Error on long channel weighted avg flux is: {err_18}; {(err_18/weighted_mean_18)*100}")

     #      ratio = weighted_mean_054/weighted_mean_18
     #      ratio_err = ratio * np.sqrt((err_054/weighted_mean_054)**2 + (err_18/weighted_mean_18)**2)
     #      print(f'Error on the ratio is: {ratio_err}; {(ratio_err/ratio)*100}')

     # Calculating the temp and em for the onset of each fractional onset interval
          #inputs: short_raw, long_raw, short_backsub, long_backsub, onset_start, onset_end, bck_short_std, bck_long_std
          #Outputs: tmk_onset, em_onset, tmk_upper, tmk_lower, em_upper, em_lower



          # tmk_eighth, em_eighth = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_eighth)
          # tmk_sixth, em_sixth = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_sixth)
          tmk_quarter, em_quarter, tmk_upper_quarter, tmk_lower_quarter, em_upper_quarter, em_lower_quarter  = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_quarter, bck_short_std, bck_long_std)
          # tmk_third, em_third = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_third)
          # tmk_half, em_half = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_half)
          # tmk_twothirds, em_twothirds = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_twothirds)
          # tmk_threequart, em_threequart = onsets.onset_tem(short_backsub, long_backsub, onset_start, endt_threequart)

          print("")
          print(f"Onset Temperature: {tmk_quarter}")
          print(f'Upper bound:{tmk_upper_quarter}; (+{tmk_upper_quarter - tmk_quarter})')
          print(f'lower bound: {tmk_lower_quarter}; (-{tmk_quarter - tmk_lower_quarter})')
     
     #Calculating error on TEM
          # print("")
          # temp_err_red = tmk_quarter*(ratio_err/ratio)**2
          # temp_err = tmk_quarter*np.sqrt((0.2)**2 + (ratio_err/ratio)**2)
          # print(f"Temp error is: {temp_err}; {(temp_err/tmk_quarter)*100}")
          # print(f"Reduced temp err is: {temp_err_red}")

          # EM_err_red = em_quarter * (err_18/weighted_mean_18)**2
          # EM_err = em_quarter * np.sqrt((0.2)**2 + (err_18/weighted_mean_18)**2)
          # print(f"EM error is: {EM_err}; {(EM_err/em_quarter)*100}")
          # print(f"Reduced EM err is: {EM_err_red}")
          # print("")


     # # Using my 'fancy' method
     #      onset_start_fancy, onset_end_fancy = onsets.onset_times(short_backsub, bck_endt, true_peak, 5*bck_short_std)
     #      try:
     #           timedelta_fancy = (onset_end_fancy - onset_start_fancy).total_seconds()
     #      except:
     #           timedelta_fancy = np.nan

     #      trunc_054_tem_fancy = np.mean(short_backsub.truncate(onset_start_fancy, onset_end_fancy))
     #      trunc_18_tem_fancy = np.mean(long_backsub.truncate(onset_start_fancy, onset_end_fancy))
     #      tmk_mnfancy, em_mnfancy = gtem.get_tem(trunc_18_tem_fancy, trunc_054_tem_fancy)

     # # Calculating temperature at flare peak (long channel)
     #      trunc_054_tem_peakl = short_backsub[true_peak]
     #      trunc_18_tem_peakl = long_backsub[true_peak]
     #      tmk_peakl, em_peakl = gtem.get_tem(trunc_18_tem_peakl, trunc_054_tem_peakl)

     # #Calculating temperature at flare peak (short_channel)
     #      trunc_054_tem_peaks = short_backsub[short_peakt]
     #      trunc_18_tem_peaks = long_backsub[short_peakt]
     #      tmk_peaks, em_peaks = gtem.get_tem(trunc_18_tem_peaks, trunc_054_tem_peaks)

     # # Calculating max flare temperature
     #      trunc_054_tem_max = short_backsub.truncate(onset_end_fancy, Time(true_peak).datetime)
     #      trunc_18_tem_max = long_backsub.truncate(onset_end_fancy, Time(true_peak).datetime)
     #      tmk_maxx, em_maxx = gtem.get_tem(trunc_18_tem_max, trunc_054_tem_max)
     
     except ValueError: 
          print("error")
          # try:
          #      tmk_max = np.nanmax(tmk_maxx)
          #      tmk_max_index = np.idxmax(tmk_max)
          #      em_max = em_maxx[tmk_max_index]
          # except:
          #      tmk_max = np.nan
          #      em_max = np.nan

#           flare_data = pd.concat([flare_data, pd.DataFrame({'Actual Peak Time': [true_peak], 'Peak Time NoSub': [old_peak], 'Peak Time': [true_peak], 'Peak Flux': [long_peakfl_18] ,'Background End Time': [bck_endt],
#                                                             'Fancy Temp': [tmk_mnfancy],'Fancy Temp Nosub': [np.nan],'Fancy EM': [em_mnfancy], 'Fancy EM Nosub':[np.nan],'Fancy Tdelta': [timedelta_fancy], 'Peak Temp Long': [tmk_peakl], 'Peak EM Long': [em_peakl],'Peak Temp Short': [tmk_peaks], 'Peak EM Short': [em_peaks], 'Flare Max Temp': [tmk_max],
#                                                             'Flare Max EM': [em_max],'Temp 1/8': [tmk_eighth], 'EM 1/8': [em_eighth],'Tdelta 1/8': [timedelta_eighth],'Temp 1/6': [tmk_sixth], 'EM 1/6': [em_sixth],'Tdelta 1/6': [timedelta_sixth],'Temp 1/4': [tmk_quarter],
#                                                             'EM 1/4': [em_quarter], 'Tdelta 1/4': [timedelta_quarter],'Temp 1/3': [tmk_third], 'EM 1/3': [em_third],'Tdelta 1/3':[timedelta_third],'Temp 1/2': [tmk_half], 'EM 1/2': [em_half],'Tdelta 1/2': [timedelta_half],'Temp 2/3': [tmk_twothirds], 
#                                                             'EM 2/3': [em_twothirds],'Tdelta 2/3': [timedelta_twothirds], 'Temp 3/4': [tmk_threequart], 'EM 3/4': [em_threequart], 'Tdelta 3/4': [timedelta_threequart], 'Background Flag': [bck_flag], 'Onset Flag': [np.nan]})], ignore_index=True)


#      except ValueError:
#           print("---------------------------------------")
#           print("Unexpected error:", sys.exc_info()[0])
#           print("ERROR, NAN ROW PLACED")
#           print("---------------------------------------")
#           flare_data = pd.concat([flare_data, pd.DataFrame({'Actual Peak Time': [np.nan], 'Peak Time NoSub': [np.nan], 'Peak Time': [np.nan],'Peak Flux': [np.nan],'Background End Time': [np.nan],
#                                                             'Fancy Temp': [np.nan], 'Fancy Temp Nosub':[np.nan],'Fancy EM': [np.nan],'Fancy EM Nosub': [np.nan],'Fancy Tdelta': [np.nan], 'Peak Temp Long': [np.nan], 'Peak EM Long': [np.nan],'Peak Temp Short': [np.nan], 'Peak EM Short': [np.nan], 'Flare Max Temp': [np.nan],
#                                                             'Flare Max EM': [np.nan],'Temp 1/8': [np.nan], 'EM 1/8': [np.nan],'Tdelta 1/8': [np.nan],'Temp 1/6': [np.nan], 'EM 1/6': [np.nan],'Tdelta 1/6': [np.nan],'Temp 1/4': [np.nan],
#                                                             'EM 1/4': [np.nan], 'Tdelta 1/4': [np.nan],'Temp 1/3': [np.nan], 'EM 1/3': [np.nan],'Tdelta 1/3':[np.nan],'Temp 1/2': [np.nan], 'EM 1/2': [np.nan],'Tdelta 1/2': [np.nan],'Temp 2/3': [np.nan], 
#                                                             'EM 2/3': [np.nan],'Tdelta 2/3': [np.nan], 'Temp 3/4': [np.nan], 'EM 3/4': [np.nan], 'Tdelta 3/4': [np.nan], 'Background Flag': [np.nan], 'Onset Flag': [np.nan]})], ignore_index=True)
#           continue

# if len(flare_data) == len(flares_df):
#      full_data = pd.concat([flares_df, flare_data],axis = 1)
# else:
#      print("ERROR, data lengths do not match")

# full_data.to_csv("2016_Flare_Data_Processed_9_avg.csv")

------------------------------
N/A Flare With Start Time: 2010-11-05 13:05:00
Background Start: 2010-11-05 13:06:40, Background End: 2010-11-05 13:07:40
Onset start: 2010-11-05 13:07:40, 1/4 Onset End: 2010-11-05 13:13:05.127750

Onset Temperature: 9.199024553213112
Upper bound:9.730312149320683; (+0.5312875961075711)
lower bound: 8.633567244988646; (-0.5654573082244667)
------------------------------
N/A Flare With Start Time: 2011-02-14 01:31:00
Background Start: 2011-02-14 01:31:10, Background End: 2011-02-14 01:32:10
Onset start: 2011-02-14 01:32:22.791000, 1/4 Onset End: 2011-02-14 01:33:24.876250

Onset Temperature: 14.857210495281862
Upper bound:15.438634250257595; (+0.5814237549757326)
lower bound: 14.260958355274079; (-0.5962521400077829)
------------------------------
N/A Flare With Start Time: 2012-05-14 13:33:00
Background Start: 2012-05-14 13:34:10, Background End: 2012-05-14 13:35:10
Onset start: 2012-05-14 13:35:11.311000, 1/4 Onset End: 2012-05-14 13:35:58.967000

Onset